## Join the Duet Server the Data Owner 1 connected to

In [2]:
import syft as sy
duet1 = sy.join_duet(loopback=True)

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > CONNECTED!


## Join the Duet Server the Data Owner 2 connected to

In [5]:
duet2 = sy.join_duet(loopback=True)

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > CONNECTED!


In [ ]:
duet1.store.pandas

In [ ]:
duet2.store.pandas

## Linear regression

In [ ]:
data1_ptr = duet1.store[0]
data2_ptr = duet2.store[0]

print(data1_ptr)
print(data2_ptr)

### Create Base Model

In [ ]:
import torch

In [ ]:
in_dim = 1
out_dim = 1

In [ ]:
class SyNet(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet, self).__init__(torch_ref=torch_ref)
        self.linear = self.torch_ref.nn.Linear(in_dim, out_dim)

    def forward(self, x):
        x = self.linear(x)
        return x

### Training

In [ ]:
def train(iterations, model, torch_ref, optim, data_ptr, target_ptr):

    losses = []

    for i in range(iterations):

        optim.zero_grad()

        output = model(data_ptr)

        loss = torch_ref.nn.functional.mse_loss(output, target_ptr)

        loss_item = loss.item()

        loss_value = loss_item.get(
            reason="To evaluate training progress",
            request_block=True,
            timeout_secs=5,
        )

        print("Epoch", i, "loss", loss_value)

        losses.append(loss_value)

        loss.backward()

        optim.step()

    return losses

#### Send one copy of the model to each data owner or client and train them remotely one by one

In [ ]:
import torch as th
import numpy as np

In [ ]:
base_model = SyNet(torch)

Train on Data Owner 1 data

In [ ]:
remote_model1 = base_model.send(duet1)

In [ ]:
remote_torch1 = duet1.torch
params = remote_model1.parameters()
optim1 = remote_torch1.optim.Adam(params=params, lr=0.1)

Dummy target data

In [ ]:
target1_ptr = th.FloatTensor(np.array([5, 10, 15, 22, 30, 38]).reshape(-1, 1))
target1_ptr

In [ ]:
iteration = 10
losses = train(iteration, remote_model1, remote_torch1, optim1, data1_ptr, target1_ptr)

Train on Data Owner 2 data

In [16]:
remote_model2 = base_model.send(duet2)

> Sending local model
> Creating remote model
  Sending local layer: linear

> Finished sending local model <




In [17]:
remote_torch2 = duet2.torch
params = remote_model2.parameters()
optim2 = remote_torch2.optim.Adam(params=params, lr=0.1)

Dummy Target data

In [18]:
target2_ptr = th.FloatTensor(np.array([35, 40, 45, 55, 60]).reshape(-1, 1))
target2_ptr

tensor([[35.],
        [40.],
        [45.],
        [55.],
        [60.]])

In [19]:
iteration = 10
losses = train(iteration, remote_model2, remote_torch2, optim2, data2_ptr, target2_ptr)

Epoch 0 loss 4783.2255859375
Epoch 1 loss 3764.81591796875
Epoch 2 loss 2872.36083984375
Epoch 3 loss 2107.156494140625
Epoch 4 loss 1469.0284423828125
Epoch 5 loss 955.904296875
Epoch 6 loss 563.3424682617188
Epoch 7 loss 284.0714416503906
Epoch 8 loss 107.63623046875
Epoch 9 loss 20.289325714111328


### Averaging Model Updates

Ideally, there will be a coordinator server with a secure aggreagtor who will get the model updates from different clients and make an aggregation. For the case of simplicity, in this example we will make the Data Sceintist server work as the coordinator.

### Little sanity check!

In [20]:
param1 = remote_model1.parameters().get(request_block=True)
param2 = remote_model2.parameters().get(request_block=True)

print("Base Model parameters:")
print(base_model.parameters())
print()

print("Remote model1 parameters:")
print(param1)
print()

print("Remote model2 parameters:")
print(param2)

Base Model parameters:
[Parameter containing:
tensor([[-0.2793]], requires_grad=True), Parameter containing:
tensor([0.2700], requires_grad=True)]

Remote model1 parameters:
[Parameter containing:
tensor([[0.6305]], requires_grad=True), Parameter containing:
tensor([1.1781], requires_grad=True)]

Remote model2 parameters:
[Parameter containing:
tensor([[0.6246]], requires_grad=True), Parameter containing:
tensor([1.1733], requires_grad=True)]


As you can see, the remote model paramter values are different from the base model paramter values. That means the remote copies of our base model got trained and updated.

In [21]:
remote_model1_updates = remote_model1.get(
    request_block=True
).state_dict()

print(remote_model1_updates)

> Downloading remote model
> Creating local model

> Finished downloading remote model <


> Saving model weights
> Finished saving weights
OrderedDict([('linear.weight', tensor([[0.6305]])), ('linear.bias', tensor([1.1781]))])


In [22]:
remote_model2_updates = remote_model2.get(
    request_block=True
).state_dict()

print(remote_model2_updates)

> Downloading remote model
> Creating local model

> Finished downloading remote model <


> Saving model weights
> Finished saving weights
OrderedDict([('linear.weight', tensor([[0.6246]])), ('linear.bias', tensor([1.1733]))])


In [23]:
from collections import OrderedDict

Let's do the aggregation of the weights. In this example, we will just calculate the average of corresponding weights from each model.

In [24]:
avg_updates = OrderedDict()
avg_updates["linear.weight"] = (
    remote_model1_updates["linear.weight"] + remote_model2_updates["linear.weight"]
) / 2
avg_updates["linear.bias"] = (
    remote_model1_updates["linear.bias"] + remote_model2_updates["linear.bias"]
) / 2

print(avg_updates)

OrderedDict([('linear.weight', tensor([[0.6276]])), ('linear.bias', tensor([1.1757]))])


### Load aggregated weights

In [25]:
combined_model = SyNet(torch)

> Creating local model


In [26]:
combined_model.load_state_dict(avg_updates)

> Loading model weights
linear state dict loaded with: <All keys matched successfully>
> Finished loading weights


In [27]:
del avg_updates, remote_model1_updates, remote_model2_updates

In [28]:
test_data = th.FloatTensor(np.array([17, 25, 32, 50, 80]).reshape(-1, 1))
test_target = th.FloatTensor(np.array([12, 15, 20, 30, 50]).reshape(-1, 1))

In [29]:
preds = []
with torch.no_grad():
    for i in range(len(test_data)):
        sample = test_data[i]
        y_hat = combined_model(sample)

        print(f"Prediction: {y_hat.item()} Ground Truth: {test_target[i].item()}")
        preds.append(y_hat)

Prediction: 11.84467887878418 Ground Truth: 12.0
Prediction: 16.86537742614746 Ground Truth: 15.0
Prediction: 21.25848960876465 Ground Truth: 20.0
Prediction: 32.5550651550293 Ground Truth: 30.0
Prediction: 51.382686614990234 Ground Truth: 50.0


In [30]:
preds = []
with torch.no_grad():
    for i in range(len(test_data)):
        sample = test_data[i]
        y_hat = base_model(sample)

        print(f"Prediction: {y_hat.item()} Ground Truth: {test_target[i].item()}")
        preds.append(y_hat)

Prediction: -4.478724956512451 Ground Truth: 12.0
Prediction: -6.713418483734131 Ground Truth: 15.0
Prediction: -8.66877555847168 Ground Truth: 20.0
Prediction: -13.696836471557617 Ground Truth: 30.0
Prediction: -22.076936721801758 Ground Truth: 50.0


## Comparison to classical linear regression on centralised data

In [ ]:
import torch
import numpy as np

in_dim = 1
out_dim = 1


class ClassicalLR(torch.nn.Module):
    def __init__(self, torch):
        super(ClassicalLR, self).__init__()
        self.linear = torch.nn.Linear(in_dim, out_dim)

    def forward(self, x):
        x = self.linear(x)
        return x


classical_model = ClassicalLR(torch)

In [ ]:
data = torch.FloatTensor(
    np.array([5, 15, 25, 35, 45, 55, 60, 65, 75, 85, 95]).reshape(-1, 1)
)
target = torch.FloatTensor(
    np.array([5, 10, 15, 22, 30, 38, 35, 40, 45, 55, 60]).reshape(-1, 1)
)

In [ ]:
def classic_train(iterations, model, torch, optim, data, target, criterion):

    losses = []

    for i in range(iterations):

        optim.zero_grad()

        output = model(data)

        loss = criterion(output, target)

        loss_item = loss.item()

        if i % 10 == 0:
            print("Epoch", i, "loss", loss_item)

        losses.append(loss_item)

        loss.backward()

        optim.step()

    return losses

In [ ]:
params = classical_model.parameters()
optim = torch.optim.Adam(params=params, lr=0.1)
criterion = torch.nn.MSELoss()

In [ ]:
iteration = 100
losses = classic_train(
    iteration, classical_model, torch, optim, data, target, criterion
)

In [ ]:
test_data = th.FloatTensor(np.array([17, 25, 32, 50, 80]).reshape(-1, 1))
test_target = th.FloatTensor(np.array([12, 15, 20, 30, 50]).reshape(-1, 1))

In [ ]:
preds = []
with torch.no_grad():
    for i in range(len(test_data)):
        sample = test_data[i]
        y_hat = classical_model(sample)

        print(f"Prediction: {y_hat.item()} Ground Truth: {test_target[i].item()}")
        preds.append(y_hat)